In [32]:
import numpy as np
from hmmlearn import hmm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

conclusion: 

Feature Selection: CCI, ADX, Chaikin A/D
Model Selection: best using SVC with poly kernel
Hyperparameter Tuning: d = 2 c = 0.3

Report

1. SVM -> stock price -> goblok tebak all up or down
2. Data collection -> jelasin features dari alphavantage + feature buatan sendiri (short SMA / long SMA ratio)
3. Model Selection -> analyze correlation  bilkang not linearly separable ...
4. Feature Selection
5. Hyperparameter tuning

stevan: 
- feature buatan sendiri (short SMA / long SMA ratio)
- training data selection
- improve acc to >70%

In [33]:
'''
CONST
'''
FILEPATH_STOCK_PRICE = "./tr_eikon_eod_data.csv"
STOCK_NAMES = ['AAPL.O', 'MSFT.O', 'INTC.O', 'AMZN.O', 'GS.N']

In [34]:
''' 
UTILS
'''
def sequential_train_test_split(data, ratio = 0.8):
    df = pd.DataFrame(data.dropna()).to_numpy()
    split_index = int(len(df) * ratio)
    return df[:split_index], df[split_index:]

def calculate_return(prediction, data) :
    data = data[:,0]
    prediction = pd.DataFrame(prediction)
    temp = pd.DataFrame(data)
    temp['log_return'] = np.log(temp / temp.shift(1))
    temp['returne'] = prediction.shift(1) * temp['log_return'].values.reshape(-1, 1)
    temp.dropna(inplace = True) 

    return np.exp(temp[['log_return', 'returne']].sum())


In [35]:
'''
BENCHMARK
'''

def benchmark_accuracy(prediction, test):
    prediction = pd.DataFrame(prediction)
    test = pd.DataFrame(test)

    return (prediction == test).sum() / len(prediction)


In [36]:
''' 
MODEL
'''

def predict_position(train, test):
    # Train Hidden Markov Model
    model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=1000)
    model.fit(train.reshape(-1, 1))

    # Predict hidden states for test data
    hidden_states = model.predict(test.reshape(-1, 1))

    # Define a function to map hidden states to rise or fall
    def map_to_rise_or_fall(state):
        return 1 if state == 0 else -1

    # Map hidden states to rise or fall
    predicted_rise_fall = np.array(list(map(map_to_rise_or_fall, hidden_states)))

    return predicted_rise_fall

In [37]:
apple_data = pd.read_csv('apple_output.csv', index_col = 0, parse_dates = True)
apple_data['SMA_EMA_ratio'] = apple_data['SMA'] / apple_data['EMA']
SMA_short = 20
SMA_long = 50
apple_data['SMA_Short'] = apple_data['AAPL.O'].rolling(SMA_short).mean()
apple_data['SMA_Long'] = apple_data['AAPL.O'].rolling(SMA_long).mean()
apple_data['SMA_Long_Short'] = apple_data['SMA_Short'] / apple_data['SMA_Long']
apple_data['direction'] = np.where(apple_data['AAPL.O'] <= apple_data['AAPL.O'].shift(-1), 1, -1)

apple_data

,AAPL.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,SMA_Short,SMA_Long,SMA_Long_Short,direction
Date,,,,,,,,,,,,,,,,,
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2010-04-01,30.572827,78.7484,82.8722,75.0702,38.0954,96.8893,42.8571,100.0000,8.816356e+10,5.524930e+10,6.9552,6.9908,0.994908,NaN,NaN,NaN,1
2010-05-01,30.625684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2010-06-01,30.138541,89.5060,82.1665,57.5986,35.3551,151.4643,57.1429,100.0000,8.681483e+10,6.043320e+10,7.5207,7.6163,0.987448,NaN,NaN,NaN,-1
2010-07-01,30.082827,11.7287,14.7644,38.9499,24.7259,-152.8759,100.0000,42.8571,8.390127e+10,5.897666e+10,8.1018,7.9595,1.017878,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,182.170000,24.7765,28.7326,33.5305,28.2742,-132.0057,100.0000,14.2857,9.108196e+10,7.277260e+10,44.7743,44.5714,1.004552,189.2060,NaN,NaN,1
2018-06-26,184.430000,30.9769,28.9171,32.5318,28.2014,-70.9622,92.8571,7.1429,9.118024e+10,7.276743e+10,44.5744,44.3849,1.004269,189.0325,NaN,NaN,-1
2018-06-27,184.160000,37.4891,31.0808,42.8627,27.4502,-43.5508,85.7143,0.0000,9.107910e+10,7.267438e+10,44.4033,44.3292,1.001672,188.8655,NaN,NaN,1


In [28]:
scaler = MinMaxScaler()

# Normalize the selected columns
normalized_columns = scaler.fit_transform(apple_data.iloc[:, :-1])

# Create a new DataFrame with the normalized values
normalized_df = pd.DataFrame(normalized_columns, columns=apple_data.columns[:-1])

# Display the normalized DataFrame
apple_data.iloc[:, :-1] = normalized_df
apple_data

,AAPL.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,SMA_Short,SMA_Long,SMA_Long_Short,direction
Date,,,,,,,,,,,,,,,,,
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2010-04-01,0.018837,0.797082,0.851941,0.769280,0.413634,0.658869,0.428571,1.000000,0.466485,0.197673,0.021918,0.021476,0.436146,NaN,NaN,NaN,1
2010-05-01,0.019154,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2010-06-01,0.016229,0.907128,0.844247,0.536156,0.374250,0.748379,0.571429,1.000000,0.417465,0.386577,0.033742,0.034488,0.269045,NaN,NaN,NaN,-1
2010-07-01,0.015895,0.111497,0.109400,0.287327,0.221484,0.249224,1.000000,0.428571,0.311572,0.333500,0.045891,0.041627,0.950701,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,0.929088,0.244971,0.261688,0.215016,0.272481,0.283454,1.000000,0.142857,0.572554,0.836229,0.812646,0.803231,0.652194,1.000000,NaN,NaN,1
2018-06-26,0.942658,0.308398,0.263699,0.201690,0.271435,0.383573,0.928571,0.071429,0.576126,0.836040,0.808467,0.799351,0.645859,0.998909,NaN,NaN,-1
2018-06-27,0.941037,0.375016,0.287289,0.339535,0.260639,0.428531,0.857143,0.000000,0.572450,0.832650,0.804889,0.798193,0.587664,0.997860,NaN,NaN,1


In [22]:
# apple_data = apple_data[['SlowK', 'RSI', 'CCI', 'Aroon Down', 'OBV', 'direction']]

In [29]:
train_data, test_data = sequential_train_test_split(apple_data)
print(test_data)
X_train, y_train, X_test, y_test = train_data[:, :-1], train_data[:, -1], test_data[:, :-1], test_data[:, -1]

[[ 0.4954496   0.74093605  0.52335572  0.40232061  0.48611285  0.58368339
   0.785714    0.          0.48867903  0.73163563  0.40465458  0.40298635
   0.48831663  0.51250111  0.59113481  0.14820252  1.        ]
 [ 0.49611008  0.91388097  0.72101672  0.39739705  0.46384454  0.60156382
   0.714286    0.          0.49281202  0.73125891  0.40523164  0.40360001
   0.48671037  0.51044578  0.59229302  0.10197966  1.        ]
 [ 0.50613745  0.95218269  0.88362859  0.42339053  0.41889831  0.67607139
   0.642857    0.          0.4970665   0.73440429  0.40525255  0.40445289
   0.45153788  0.5085853   0.59334427  0.06020329  1.        ]
 [ 0.50655775  0.93761163  0.95349897  0.53054945  0.37165684  0.7020409
   0.571429    1.          0.50084136  0.73358796  0.40604915  0.40675777
   0.38813259  0.50655826  0.59398772  0.0206272  -1.        ]
 [ 0.50313524  0.84991827  0.93077613  0.48969854  0.34350311  0.62546842
   0.5         0.928571    0.49685412  0.73568846  0.4067663   0.40812654
   0.3609

In [30]:
X_train

array([[0.09863088, 0.2787488 , 0.38937689, 0.39794545, 0.20078559,
        0.322164  , 0.214286  , 0.714286  , 0.56465215, 0.51701562,
        0.07537185, 0.07307981, 0.64553204, 0.08975235, 0.        ,
        0.86821288],
       [0.09396031, 0.18193061, 0.26684343, 0.34684045, 0.22223324,
        0.24239509, 1.        , 0.642857  , 0.54074988, 0.50619084,
        0.07529031, 0.07221444, 0.73550903, 0.08939767, 0.00104528,
        0.81811984],
       [0.09302534, 0.14228779, 0.1716816 , 0.27189952, 0.24615004,
        0.27957531, 1.        , 0.571429  , 0.52332429, 0.50408755,
        0.07464843, 0.07099128, 0.80384217, 0.08894737, 0.00194253,
        0.76919325],
       [0.09982747, 0.27072776, 0.1688328 , 0.36557668, 0.23238002,
        0.42618604, 0.928571  , 0.5       , 0.54129657, 0.51313346,
        0.07400028, 0.07044627, 0.7930471 , 0.08889843, 0.00302553,
        0.73130281],
       [0.09836926, 0.47026251, 0.27126437, 0.42441127, 0.2199883 ,
        0.42702791, 0.857143  , 

In [31]:
from sklearn.svm import SVC
model = SVC(kernel='rbf', gamma=0.7, C=1.0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
benchmark_accuracy(pred, y_test)

0    0.384615
dtype: float64

In [29]:
best_acc = 0
best_g = -1
best_c = -1

selected_features = [4, 5, 9]

for g in range(1, 11, 1):
    for c in range(1, 51, 1):
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='rbf', gamma=g / 10, C=c / 10)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        print(f"acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_g = g
            best_c = c
        print(f"best acc: {best_acc}, d: {best_g}, c: {best_c}")
    
print(f"best acc: {best_acc}, g: {best_g}, c: {best_c}")

acc: 0.519 up: 0.0
best acc: 0.5185185185185185, d: 1, c: 1
acc: 0.519 up: 0.0
best acc: 0.5185185185185185, d: 1, c: 1
acc: 0.519 up: 0.0
best acc: 0.5185185185185185, d: 1, c: 1
acc: 0.519 up: 0.0
best acc: 0.5185185185185185, d: 1, c: 1
acc: 0.519 up: 0.0
best acc: 0.5185185185185185, d: 1, c: 1
acc: 0.521 up: 0.003
best acc: 0.5211640211640212, d: 1, c: 6
acc: 0.521 up: 0.003
best acc: 0.5211640211640212, d: 1, c: 6
acc: 0.519 up: 0.005
best acc: 0.5211640211640212, d: 1, c: 6
acc: 0.524 up: 0.016
best acc: 0.5238095238095238, d: 1, c: 9
acc: 0.526 up: 0.019
best acc: 0.5264550264550265, d: 1, c: 10
acc: 0.537 up: 0.045
best acc: 0.5370370370370371, d: 1, c: 11
acc: 0.545 up: 0.058
best acc: 0.544973544973545, d: 1, c: 12
acc: 0.548 up: 0.061
best acc: 0.5476190476190477, d: 1, c: 13
acc: 0.556 up: 0.079
best acc: 0.5555555555555556, d: 1, c: 14
acc: 0.561 up: 0.09
best acc: 0.5608465608465608, d: 1, c: 15
acc: 0.563 up: 0.098
best acc: 0.5634920634920635, d: 1, c: 16
acc: 0.553 up

KeyboardInterrupt: 

In [151]:
best_acc = 0
best_c = -1

selected_features = [4, 5, 9]

for c in range(1, 101, 1):
    curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
    curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
    model = SVC(kernel='sigmoid', C= c/10)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    print(f"acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")

    if acc[0] > best_acc:
        best_acc = acc[0]
        best_g = g
        best_c = c
    print(f"best acc: {best_acc}, c: {best_c}")
    
print(f"best acc: {best_acc}, c: {best_c}")

acc: 0.519 up: 0.0
best acc: 0.5185185185185185, c: 1
acc: 0.519 up: 0.0
best acc: 0.5185185185185185, c: 1
acc: 0.521 up: 0.198
best acc: 0.5211640211640212, c: 3
acc: 0.532 up: 0.098
best acc: 0.5317460317460317, c: 4
acc: 0.542 up: 0.743
best acc: 0.5423280423280423, c: 5
acc: 0.537 up: 0.749
best acc: 0.5423280423280423, c: 5
acc: 0.503 up: 0.037
best acc: 0.5423280423280423, c: 5
acc: 0.489 up: 0.071
best acc: 0.5423280423280423, c: 5
acc: 0.495 up: 0.082
best acc: 0.5423280423280423, c: 5
acc: 0.487 up: 0.101
best acc: 0.5423280423280423, c: 5
acc: 0.484 up: 0.108
best acc: 0.5423280423280423, c: 5
acc: 0.497 up: 0.132
best acc: 0.5423280423280423, c: 5
acc: 0.481 up: 0.148
best acc: 0.5423280423280423, c: 5
acc: 0.481 up: 0.159
best acc: 0.5423280423280423, c: 5
acc: 0.468 up: 0.172
best acc: 0.5423280423280423, c: 5
acc: 0.46 up: 0.18
best acc: 0.5423280423280423, c: 5
acc: 0.463 up: 0.188
best acc: 0.5423280423280423, c: 5
acc: 0.463 up: 0.204
best acc: 0.5423280423280423, c: 

In [136]:
best_acc = 0
best_d = -1
best_c = -1

selected_features = [4, 5, 9]

for d in range(1, 5, 1):
    for c in range(1, 51, 1):
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree=d, C=c/10)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        print(f"acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_d = d
            best_c = c
        print(f"best acc: {best_acc}, d: {best_d}, c: {best_c}")
    
print(f"best acc: {best_acc}, d: {best_d}, c: {best_c}")

acc: 0.595 up: 0.177
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.579 up: 0.246
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.577 up: 0.243
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.569 up: 0.262
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.5

In [54]:
best_acc = 0
best_c = -1

selected_features = [4, 5, 9]

# for d in range(1, 5, 1):
for c in range(1, 101, 1):
    curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
    curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
    model = SVC(kernel='poly', degree=2, C = c / 10)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    print(f"acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")

    if acc[0] > best_acc:
        best_acc = acc[0]
        best_c = c
    print(f"best acc: {best_acc}, c: {best_c}")
    
print(f"best acc: {best_acc}, c: {best_c}")

acc: 0.619 up: 0.28
best acc: 0.6190476190476191, c: 1
acc: 0.611 up: 0.257
best acc: 0.6190476190476191, c: 1
acc: 0.622 up: 0.246
best acc: 0.6216931216931217, c: 3
acc: 0.606 up: 0.235
best acc: 0.6216931216931217, c: 3
acc: 0.608 up: 0.238
best acc: 0.6216931216931217, c: 3
acc: 0.606 up: 0.235
best acc: 0.6216931216931217, c: 3
acc: 0.606 up: 0.235
best acc: 0.6216931216931217, c: 3
acc: 0.608 up: 0.233
best acc: 0.6216931216931217, c: 3
acc: 0.608 up: 0.233
best acc: 0.6216931216931217, c: 3
acc: 0.608 up: 0.228
best acc: 0.6216931216931217, c: 3
acc: 0.608 up: 0.228
best acc: 0.6216931216931217, c: 3
acc: 0.611 up: 0.23
best acc: 0.6216931216931217, c: 3
acc: 0.611 up: 0.23
best acc: 0.6216931216931217, c: 3
acc: 0.611 up: 0.23
best acc: 0.6216931216931217, c: 3
acc: 0.611 up: 0.23
best acc: 0.6216931216931217, c: 3
acc: 0.611 up: 0.23
best acc: 0.6216931216931217, c: 3
acc: 0.611 up: 0.23
best acc: 0.6216931216931217, c: 3
acc: 0.611 up: 0.23
best acc: 0.6216931216931217, c: 3


In [ ]:
# Feature Engineering

# Feature Selection

In [43]:
X = apple_data.dropna().iloc[:, :-1]
y = apple_data.dropna().iloc[:, -1]

In [44]:
X

,AAPL.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio
Date,,,,,,,,,,,,,
2010-04-01,0.018837,0.797082,0.851941,0.769280,0.413634,0.658869,0.428571,1.000000,0.466485,0.197673,0.021918,0.021476,0.436146
2010-06-01,0.016229,0.907128,0.844247,0.536156,0.374250,0.748379,0.571429,1.000000,0.417465,0.386577,0.033742,0.034488,0.269045
2010-07-01,0.015895,0.111497,0.109400,0.287327,0.221484,0.249224,1.000000,0.428571,0.311572,0.333500,0.045891,0.041627,0.950701
2010-11-01,0.015491,0.258310,0.233819,0.445504,0.149730,0.329648,0.000000,0.285714,0.566643,0.506703,0.070674,0.068125,0.680743
2010-12-01,0.013441,0.723159,0.787620,0.558717,0.137354,0.660432,0.357143,0.000000,0.561133,0.530421,0.072551,0.072439,0.398836
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,0.929088,0.244971,0.261688,0.215016,0.272481,0.283454,1.000000,0.142857,0.572554,0.836229,0.812646,0.803231,0.652194
2018-06-26,0.942658,0.308398,0.263699,0.201690,0.271435,0.383573,0.928571,0.071429,0.576126,0.836040,0.808467,0.799351,0.645859
2018-06-27,0.941037,0.375016,0.287289,0.339535,0.260639,0.428531,0.857143,0.000000,0.572450,0.832650,0.804889,0.798193,0.587664


In [45]:
y

Date
2010-04-01    1
2010-06-01   -1
2010-07-01    1
2010-11-01   -1
2010-12-01    1
             ..
2018-06-25    1
2018-06-26   -1
2018-06-27    1
2018-06-28   -1
2018-06-29   -1
Name: direction, Length: 1887, dtype: int64

In [46]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Assuming 'X' contains the features and 'y' contains the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Selecting the most relevant features using mutual information
selector = SelectKBest(score_func=mutual_info_classif, k=5)  # Select top 5 features
X_train_selected = selector.fit_transform(X_train, y_train)
selected_features = X.columns[selector.get_support()]

# Train a model using the selected features
model = LogisticRegression()
model.fit(X_train_selected, y_train)

# Evaluate the model on the test set
X_test_selected = selector.transform(X_test)
accuracy = model.score(X_test_selected, y_test)

# Display the most relevant features and model accuracy
print("Selected features:", selected_features)
print("Model accuracy:", accuracy)

Selected features: Index(['SlowK', 'RSI', 'CCI', 'OBV', 'EMA'], dtype='object')
Model accuracy: 0.5238095238095238


In [52]:
best_acc = 0

for i in range(len(apple_data.columns) - 1):
    curr_X_train = X_train[:, 1].reshape(-1, 1)
    curr_X_test = X_test[:, i].reshape(-1, 1)
    model = SVC(kernel='rbf', gamma= 1.0, C= 1.0)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    print(f"feature {i}: acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")

feature 0: acc: 0.474 up: 0.622
feature 1: acc: 0.524 up: 0.423
feature 2: acc: 0.521 up: 0.415
feature 3: acc: 0.508 up: 0.159
feature 4: acc: 0.534 up: 0.063
feature 5: acc: 0.553 up: 0.146
feature 6: acc: 0.521 up: 0.214
feature 7: acc: 0.511 up: 0.484
feature 8: acc: 0.519 up: 0.0
feature 9: acc: 0.481 up: 0.937
feature 10: acc: 0.46 up: 0.302
feature 11: acc: 0.476 up: 0.238
feature 12: acc: 0.524 up: 0.074


In [59]:
best_acc = 0
best_selected_features = []

for i in range(len(apple_data.columns) - 1):
    for j in range(len(apple_data.columns) - 1):
        selected_features = [i, j]
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree = 2, C= 0.3)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        print(f"feature {i}: acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_selected_features = selected_features
        print(f"best acc: {best_acc} features: {best_selected_features}")

feature 0: acc: 0.519 up: 0.0
best acc: 0.5185185185185185 features: [0, 0]
feature 0: acc: 0.521 up: 0.008
best acc: 0.5211640211640212 features: [0, 1]
feature 0: acc: 0.519 up: 0.0
best acc: 0.5211640211640212 features: [0, 1]
feature 0: acc: 0.521 up: 0.003
best acc: 0.5211640211640212 features: [0, 1]
feature 0: acc: 0.519 up: 0.0
best acc: 0.5211640211640212 features: [0, 1]
feature 0: acc: 0.519 up: 0.005
best acc: 0.5211640211640212 features: [0, 1]
feature 0: acc: 0.519 up: 0.0
best acc: 0.5211640211640212 features: [0, 1]
feature 0: acc: 0.521 up: 0.003
best acc: 0.5211640211640212 features: [0, 1]
feature 0: acc: 0.519 up: 0.0
best acc: 0.5211640211640212 features: [0, 1]
feature 0: acc: 0.519 up: 0.0
best acc: 0.5211640211640212 features: [0, 1]
feature 0: acc: 0.529 up: 0.037
best acc: 0.5291005291005291 features: [0, 10]
feature 0: acc: 0.529 up: 0.037
best acc: 0.5291005291005291 features: [0, 10]
feature 0: acc: 0.519 up: 0.0
best acc: 0.5291005291005291 features: [0, 1

In [65]:
import itertools

all_features = list(range(len(apple_data.columns) - 1))
selected_features = list(itertools.permutations(all_features, 2))

best_acc = 0
best_selected_features = []

for i in range(1, len(apple_data.columns) - 1):
    possible_selected_features = list(itertools.permutations(all_features, i))
    for selected_features in possible_selected_features:
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree = 2, C= 0.3)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        print(f"feature {i}: acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}, features: {selected_features}")

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_selected_features = selected_features
        print(f"best acc: {best_acc} features: {best_selected_features}")

feature 1: acc: 0.519 up: 0.0, features: (0,)
best acc: 0.5185185185185185 features: (0,)
feature 1: acc: 0.545 up: 0.471, features: (1,)
best acc: 0.544973544973545 features: (1,)
feature 1: acc: 0.526 up: 0.458, features: (2,)
best acc: 0.544973544973545 features: (1,)
feature 1: acc: 0.503 up: 0.328, features: (3,)
best acc: 0.544973544973545 features: (1,)
feature 1: acc: 0.532 up: 0.082, features: (4,)
best acc: 0.544973544973545 features: (1,)
feature 1: acc: 0.534 up: 0.466, features: (5,)
best acc: 0.544973544973545 features: (1,)
feature 1: acc: 0.519 up: 0.0, features: (6,)
best acc: 0.544973544973545 features: (1,)
feature 1: acc: 0.503 up: 0.529, features: (7,)
best acc: 0.544973544973545 features: (1,)
feature 1: acc: 0.55 up: 0.534, features: (8,)
best acc: 0.5502645502645502 features: (8,)
feature 1: acc: 0.519 up: 0.011, features: (9,)
best acc: 0.5502645502645502 features: (8,)
feature 1: acc: 0.521 up: 0.013, features: (10,)
best acc: 0.5502645502645502 features: (8,)

KeyboardInterrupt: 

array([[0.01883667, 0.79708231],
       [0.01622904, 0.9071284 ],
       [0.01589451, 0.11149665],
       ...,
       [0.53766059, 0.68329218],
       [0.54156345, 0.72807436],
       [0.5410831 , 0.67998392]])